## Učitati skup podataka 

In [1]:
import pandas as pd
import numpy as np

In [2]:
bank=pd.read_csv('C:\\Users\\Dusica\\Downloads\\Bank Marketing\\bank-additional.txt',sep=';')

## Deskriptivna statistika skupa podataka 

In [3]:
bank.shape


(4119, 21)

In [4]:
bank.describe()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m
count,4119.000000,4116.000000,4114.000000,4119.000000,4117.000000,4116.000000,4117.000000,4117.000000,4117.000000
mean,40.113620,256.869048,2.534273,960.422190,0.190430,0.084694,93.579540,-40.486358,4.858993
std,10.313362,254.763913,2.566673,191.922786,0.541904,1.563361,0.579442,4.639840,79.432456
min,18.000000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.635000
25%,32.000000,103.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.334000
50%,38.000000,181.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000
75%,47.000000,317.250000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000
max,88.000000,3643.000000,35.000000,999.000000,6.000000,1.400000,94.767000,1.266000,5099.100000


In [5]:
bank.count()

age               4119
job               4119
marital           4119
education         4119
default           4119
housing           4119
loan              4119
contact           4119
month             4119
day_of_week       4119
duration          4116
campaign          4114
pdays             4119
previous          4117
poutcome          4119
emp.var.rate      4116
cons.price.idx    4117
cons.conf.idx     4117
euribor3m         4117
nr.employed       4118
y                 4118
dtype: int64

In [6]:
pd.concat([bank.dtypes,bank.isnull().sum()],axis=1)

,0,1
age,int64,0
job,object,0
marital,object,0
education,object,0
default,object,0
housing,object,0
loan,object,0
contact,object,0
month,object,0
day_of_week,object,0


## Priprema skupa podataka
(tako da bude pogodan za prediktivno modelovanje)

Input(duration) should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.

In [7]:
bank.drop(['duration'],axis=1,inplace=True)

### *Resavanje null vrednosti*

In [8]:
bank.dropna(subset=['y'],inplace=True)

In [9]:
#set(bank['nr.employed'])

In [10]:
pd.concat([bank['cons.price.idx'].describe(),bank['previous'].describe()],axis=1)

#moze mean jer std ima malu vrednost

,cons.price.idx,previous
count,4116.000000,4116.000000
mean,93.579707,0.190476
std,0.579413,0.541961
min,92.201000,0.000000
25%,93.075000,0.000000
50%,93.749000,0.000000
75%,93.994000,0.000000
max,94.767000,6.000000


In [11]:
mean_cons_price_idx=bank['cons.price.idx'].dropna().mean()
#mean_cons_price_idx=np.average(bank['cons.price.idx'].dropna())

In [12]:
mean_previous=bank['previous'].dropna().mean()


In [13]:
bank['cons.price.idx'].replace(np.nan,mean_cons_price_idx,inplace=True)

In [14]:
bank['previous'].replace(np.nan,mean_previous,inplace=True)

In [15]:
bank['nr.employed']=pd.to_numeric(bank['nr.employed'],downcast='float')

In [16]:
bank['nr.employed'].fillna(value=bank['nr.employed'].dropna().mean(),inplace=True)

In [17]:
bank['nr.employed'].describe()

count    4118.000000
mean     5166.518555
std        73.662903
min      4963.600098
25%      5099.100098
50%      5191.000000
75%      5228.100098
max      5228.100098
Name: nr.employed, dtype: float64

campaign	float64	5
emp.var.rate	float64	3
cons.conf.idx	float64	2
euribor3m	float64	2

In [18]:
pd.concat([bank['campaign'].describe(),bank['emp.var.rate'].describe(),bank['cons.conf.idx'].describe(),bank['euribor3m'].describe()],axis=1)

,campaign,emp.var.rate,cons.conf.idx,euribor3m
count,4113.000000,4115.000000,4116.000000,4116.000000
mean,2.533917,0.085152,-40.496501,3.621325
std,2.566883,1.563274,4.594520,1.733618
min,1.000000,-3.400000,-50.800000,0.635000
25%,1.000000,-1.800000,-42.700000,1.334000
50%,2.000000,1.100000,-41.800000,4.857000
75%,3.000000,1.400000,-36.400000,4.961000
max,35.000000,1.400000,-26.900000,5.045000


In [19]:
bank['campaign'].replace(np.nan,bank['campaign'].dropna().mean(),inplace=True)

In [20]:
bank['emp.var.rate'].replace(np.nan,bank['emp.var.rate'].dropna().mean(),inplace=True)

In [21]:
bank['cons.conf.idx'].replace(np.nan,bank['cons.conf.idx'].dropna().mean(),inplace=True)

In [22]:
bank['euribor3m'].replace(np.nan,bank['euribor3m'].dropna().mean(),inplace=True)

eliminate categorical variables from sample

In [23]:
bank_without_cat=bank.select_dtypes(exclude=['object'])

## Podela skupa podataka na trening i test u odnosu 70:30

In [24]:
def split_train_test(bank, test_percentage, shuffle=True):
    df=bank.sample(frac=1)
    n=len(df)
    split=int(np.floor(n*test_percentage))
    test= df.iloc[:split,:]
    train=df.iloc[split:,:]
    return (train, test)

In [25]:
train,test=split_train_test(bank,0.3)


In [26]:
train.shape

(2883, 20)

In [27]:
test.shape

(1235, 20)

In [28]:
y_train=train['y']
#y_train=train.loc[:,'y']

In [29]:
y_train.shape

(2883,)

In [30]:
x_train=train.drop('y',axis=1).select_dtypes(exclude=['object'])

In [31]:
x_train.shape

(2883, 9)

In [32]:
x_test=test.drop('y',axis=1).select_dtypes(exclude=['object'])

In [33]:
y_test=test['y']

Kreirati minimalno 2 prediktivna modela (sa default parametrima) i oceniti gresku na test setu (minimum 2 mere evaluacije)

In [34]:
#x_train.day_of_week.unique()

In [35]:
from sklearn.linear_model import LogisticRegression

In [36]:
lr=LogisticRegression()

In [37]:
modelLR=lr.fit(x_train,y_train)

In [38]:
predictionsLR=modelLR.predict(x_test)

In [39]:
predictionsLR

array(['no', 'no', 'no', ..., 'no', 'no', 'no'], dtype=object)

In [42]:
print(type(predictionsLR))
print(type(y_test))

<class 'numpy.ndarray'>
<class 'pandas.core.series.Series'>


In [43]:
y_real=np.array(y_test)

In [44]:
y_real_train=np.array(y_train)

In [45]:
y_real

array(['no', 'no', 'no', ..., 'no', 'no', 'no'], dtype=object)

In [46]:
predictionsLR=modelLR.predict(x_test)


In [47]:
predict_train=modelLR.predict(x_train)


In [48]:
true_predicted=np.column_stack((y_real, predictionsLR))


In [49]:
prediction_probs=modelLR.predict_proba(x_test)

In [50]:
true_predicted.shape

(1235, 2)

In [51]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [52]:
accuracy_score(y_train,predict_train)

0.90669441553936869

In [53]:
precision_score(y_train,predict_train,pos_label='yes',average="binary")
recall_score(y_train,predict_train,pos_label='yes')

0.20333333333333334

In [54]:
accuracy_score(y_test,predictionsLR)

0.89311740890688263

In [55]:
confusion_matrix(y_test, predictionsLR)

array([[1073,   11],
       [ 121,   30]], dtype=int64)

In [56]:
y_test.value_counts()

no     1084
yes     151
Name: y, dtype: int64

In [57]:
precision_score(y_test,predictionsLR,pos_label='yes',average="binary")

0.73170731707317072

In [58]:
recall_score(y_test,predictionsLR,pos_label='yes')

0.19867549668874171

In [59]:
from sklearn.ensemble import RandomForestClassifier

In [60]:
rf=RandomForestClassifier()

In [61]:
modelRF=rf.fit(x_train,y_train)

In [62]:
predictionsRF=modelRF.predict(x_test)

In [63]:
predictionsRF

array(['no', 'no', 'no', ..., 'no', 'no', 'no'], dtype=object)

In [64]:
accuracy_score(y_test,predictionsRF)

0.86396761133603239

In [65]:
confusion_matrix(y_test,predictionsRF)

array([[1035,   49],
       [ 119,   32]], dtype=int64)

In [66]:
precision_score(y_test,predictionsRF,pos_label='yes')

0.39506172839506171

In [67]:
recall_score(y_test,predictionsRF,pos_label='yes')

0.2119205298013245

### Cross validation

In [68]:
from sklearn.model_selection import cross_val_score

In [69]:
models=[lr,rf]

In [70]:
len(models)

2

In [73]:
for mod in models:
    print (mod)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


In [74]:
for x in range(len(models)):
     #print (x)
     print (type(models[-1]))

<class 'sklearn.ensemble.forest.RandomForestClassifier'>
<class 'sklearn.ensemble.forest.RandomForestClassifier'>


In [75]:
for num in range(len(models)):
    scores=cross_val_score(models[num-1],x_train,y_train,cv=5,scoring='accuracy')
    print (scores,'--->',scores.mean())

[ 0.88734835  0.88908146  0.89081456  0.87847222  0.88715278] ---> 0.886573873484
[ 0.89948007  0.90987868  0.90641248  0.90104167  0.91666667] ---> 0.906695912767


In [76]:
#np.average(scores)

## Optimizacija parametara

In [80]:
from sklearn.model_selection import GridSearchCV

In [82]:
rf

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [89]:
params = [
  {'n_estimators': [20, 40, 100, 300, 500], 'max_depth': [5, 10, 15]}
 ]

In [90]:
optimizer = GridSearchCV(rf, 
                   param_grid=params, 
                   scoring='roc_auc', cv=3,  
                   return_train_score=True, 
                   refit='roc_auc', n_jobs=2)

In [95]:
opt_rf=optimizer.fit(x_train,y_train)


JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
C:\Users\Dusica\Anaconda3\lib\runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...nda3\\lib\\site-packages\\ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
C:\Users\Dusica\Anaconda3\lib\runpy.

In [88]:
opt_rf.best_estimator_


NameError: name 'opt_rf' is not defined

In [ ]:
probas_opt=opt_rf.predict_proba(X_test)[:,1]


In [1038]:
predictions_opt=opt_rf.predict(X_test)


NameError: name 'opt_rf' is not defined

In [ ]:
roc_auc_score(y_test, predictions_opt)


### Podela inicijalnog skupa podataka na osnovu jednog kategoričkog atributa u datasetu

In [96]:
#bank.sort_values(['contact'])

In [125]:
#bank.loc[bank['loan'] == 'yes']

In [126]:
loan_yes=bank.loc[lambda bank:bank['loan']=='yes']

In [104]:
categories=bank.loan.unique()

In [128]:
for n in range(len(categories)):
        print(n)

0
1
2


In [161]:
listaDF=[pd.DataFrame,pd.DataFrame,pd.DataFrame]

In [173]:
#def vratiSS():
for n in range(len(categories)):
    listaDF[n]=bank.loc[lambda bank:bank['loan']==categories[n]]
    #return listaDF[n]

In [176]:
#listaDF[1].loan.unique()

array(['unknown'], dtype=object)

(3349, 20)

In [844]:
import seaborn as sns

In [851]:
ax1=sns.distplot(bank['y'],hist=False)

TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [760]:
type(np.array(prediction_probs))


numpy.ndarray

In [765]:
prediction_probs[1]

array([ 0.95796279,  0.04203721])

In [773]:
import numpy as np
import matplotlib.pyplot as plt

#t = np.arange(0., 1., 0.2)
x,y= np.polyfit(np.array(prediction_probs), np.array(prediction_probs), 1)
# red dashes, blue squares and green triangles
plt.plot( x, 'r--', y, 'bs')
plt.xlim(0,1)
plt.ylim(0,1)
plt.show()

TypeError: expected 1D vector for x